## (1) Packages and settings

In [1]:
import numpy as np
import pyccl as ccl

## (2) Compute the independent modes 
Code adopted from https://github.com/franciscovillaescusa/Pylians3

In [2]:
# same as frequencies but in 2D
def frequencies_2D(BoxSize,dims):
    kF = 2.0*np.pi/BoxSize;  middle = dims//2;  kN = middle*kF
    kmax_par = middle
    kmax_per = middle
    kmax     = int(np.sqrt(middle**2 + middle**2))
    return kF,kN,kmax_par,kmax_per,kmax

# This function computes all independent modes
def check_number_modes_2D(dims):
    # (0,0) own antivector, while (n,n) has (-n,-n) for dims odd
    if dims%2==1:  own_modes = 1 
    # (0,0),(0,n),(n,0),(n,n)
    else:          own_modes = 4
    repeated_modes = (dims**2 - own_modes)//2  
    indep_modes    = repeated_modes + own_modes
    
    return indep_modes

In [3]:
Npixel, BoxSize = 100, 1000
kF,kN,kmax_par,kmax_per,kmax = frequencies_2D(BoxSize,Npixel)
middle = Npixel//2
Nmodes = check_number_modes_2D(Npixel)
print(kF, kmax*kF, kmax)
print('Number of independent modes: ', check_number_modes_2D(Npixel))


0.006283185307179587 0.43982297150257105 70
Number of independent modes:  5002


In [4]:
k2D    = np.zeros(kmax+1, dtype=np.float64)
Pk2D   = np.zeros(kmax+1, dtype=np.float64)
Nmodes = np.zeros(kmax+1, dtype=np.float64)
ks = []
for kxx in range(Npixel):
    kx = (kxx-Npixel if (kxx>middle) else kxx)
    for kyy in range(middle+1):
        ky = (kyy-Npixel if (kyy>middle) else kyy)
        
        if ky==0 or (ky==middle and Npixel%2==0):
            if kx<0:  
                continue
                
        # compute magnitude |k| of the mode #and its integer part        
        k = np.sqrt(kx**2 + ky**2)
        k_index = int(k)
        ks.append(k)

k = np.array(ks[1:])*kF


## (3.1) Compute the power spectrum $P(k)$ for fiducial cosmology

In [5]:
OmegaM = 0.3
sigma8 = 0.8

OmegaB = 0.05
OmegaC = OmegaM - OmegaB

h    = 0.7
ns   = 0.96

cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                          h=h, sigma8 = sigma8, n_s=ns,
                          transfer_function='eisenstein_hu')

Pk_fid = ccl.linear_matter_power(cosmo_ccl, k, 1.0)

## (3.2) Compute the derivatives of P(k) with respect to $\Omega_M$, $\sigma_8$

In [6]:
def get_delta_OmegaM(delta, OmegaM_fid, k, sigma8=0.8, OmegaB=0.05, ns= 0.96, h=0.7):
    #####################################################
    OmegaM = OmegaM_fid + delta
    OmegaC = OmegaM - OmegaB
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                              h=h, sigma8 = sigma8, n_s=ns, 
                              transfer_function='eisenstein_hu')
    Pk_delta = ccl.linear_matter_power(cosmo_ccl, k, 1.0)
    #####################################################
    OmegaM = OmegaM_fid + 2*delta
    OmegaC = OmegaM - OmegaB
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                              h=h, sigma8 = sigma8, n_s=ns, 
                              transfer_function='eisenstein_hu')
    Pk_2delta = ccl.linear_matter_power(cosmo_ccl, k, 1.0)
    #####################################################
    OmegaM = OmegaM_fid - delta
    OmegaC = OmegaM - OmegaB
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                              h=h, sigma8 = sigma8, n_s=ns, 
                              transfer_function='eisenstein_hu')
    Pk_neg_delta = ccl.linear_matter_power(cosmo_ccl, k, 1.0)
    #####################################################
    OmegaM = OmegaM_fid - 2*delta
    OmegaC = OmegaM - OmegaB
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                              h=h, sigma8 = sigma8, n_s=ns, 
                              transfer_function='eisenstein_hu')
    Pk_neg_2delta = ccl.linear_matter_power(cosmo_ccl, k, 1.0)
    #####################################################
    deriv = (-Pk_2delta + 8*Pk_delta - 8*Pk_neg_delta + Pk_neg_2delta)/(12*delta)
    
    return deriv

def get_delta_sigma8(delta, sigma8_fid, k, OmegaM=0.3, OmegaB=0.05, ns=0.96, h=0.7):
    OmegaC = OmegaM - OmegaB
    #####################################################
    sigma8 = sigma8_fid + delta
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                              h=h, sigma8 = sigma8, n_s=ns, 
                              transfer_function='eisenstein_hu')
    Pk_delta = ccl.linear_matter_power(cosmo_ccl, k, 1.0)
    #####################################################
    sigma8 = sigma8_fid + 2*delta
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                              h=h, sigma8 = sigma8, n_s=ns, 
                              transfer_function='eisenstein_hu')
    Pk_2delta = ccl.linear_matter_power(cosmo_ccl, k, 1.0)
    #####################################################
    sigma8 = sigma8_fid - delta
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                              h=h, sigma8 = sigma8, n_s=ns, 
                              transfer_function='eisenstein_hu')
    Pk_neg_delta = ccl.linear_matter_power(cosmo_ccl, k, 1.0)
    #####################################################
    sigma8 = sigma8_fid - 2*delta
    cosmo_ccl = ccl.Cosmology(Omega_c=OmegaC, Omega_b=OmegaB, 
                              h=h, sigma8 = sigma8, n_s=ns, 
                              transfer_function='eisenstein_hu')
    Pk_neg_2delta = ccl.linear_matter_power(cosmo_ccl, k, 1.0)
    #####################################################
    deriv = (-Pk_2delta + 8*Pk_delta - 8*Pk_neg_delta + Pk_neg_2delta)/(12*delta)
    
    return deriv

## (4) Compute Fisher matrix

In [7]:
OmegaM_fid = OmegaM
delta = 2e-4*OmegaM_fid
dPdOmegaM = get_delta_OmegaM(delta, OmegaM_fid, k)

sigma8_fid = sigma8
delta = 0.05*sigma8_fid 
dPdsigma8 = get_delta_sigma8(delta, sigma8_fid, k)


In [8]:
F_AA = np.sum(dPdOmegaM*dPdOmegaM/Pk_fid**2)/2
F_BB = np.sum(dPdsigma8*dPdsigma8/Pk_fid**2)/2
F_AB = np.sum(dPdOmegaM*dPdsigma8/Pk_fid**2)/2

F_m = np.array([[F_AA, F_AB], [F_AB, F_BB]])
F_inv = np.linalg.inv(F_m)
stdevA, stdevB = np.sqrt(F_inv[0, 0]), np.sqrt(F_inv[1, 1])
print('Relative Errors on OmegaM, sigma8 in %: ', stdevA/OmegaM_fid*100, stdevB/sigma8_fid*100)
print('\nFisher matrix: \n', F_m, '\n \nInverse Fisher matrix: \n', F_inv)
print('\nFisher information: ', np.log(np.linalg.det(F_m))/2)

Relative Errors on OmegaM, sigma8 in %:  6.449145194480185 1.621833668739181

Fisher matrix: 
 [[ 7028.33826638  8251.6304471 ]
 [ 8251.6304471  15628.125     ]] 
 
Inverse Fisher matrix: 
 [[ 0.00037432 -0.00019764]
 [-0.00019764  0.00016834]]

Fisher information:  8.773609123580158
